In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from passivbot_futures import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import PrettyPrinter

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)
pp = PrettyPrinter()

In [ ]:
user = 'your_user_name'
settings = load_settings(user)
s = settings['symbol']
settings

In [ ]:
try:
    await bot.cc.close()
except:
    pass
bot = await create_bot(user, settings)
cc = bot.cc

In [ ]:
n_days_load = 7
agg_trades = await bot.load_trades(s, n_days_load)
adf = agg_trades

In [ ]:
adf.price.iloc[::100].plot()

In [ ]:
# modify settings
settings['entry_amount'] = 0.001
settings['leverage'] = 100
settings['markup'] = 0.0015
settings['flashcrash_factor'] = 0.0001
settings['ema_span'] = 1000
settings['enter_long'] = True
settings['enter_shrt'] = True
settings

In [ ]:
trades, adf = backtest(adf, settings)

tdf = pd.DataFrame(trades).set_index('agg_id')
adf.price.plot()
bdf = tdf[tdf.side == 'buy']
sdf = tdf[tdf.side == 'sel']
bdf.price.plot(style='bo')
sdf.price.plot(style='ro')
tdf

In [ ]:
realized_pnl_sum = tdf.realized_pnl.sum()
fee_sum = tdf.fee.sum()
print('realized pnl sum', realized_pnl_sum)
print('fee sum', fee_sum)
print('pnl - fee', realized_pnl_sum - fee_sum)
print('pct fees paid', fee_sum / realized_pnl_sum)

In [ ]:
print('pos amount counts')
tdf.amount.value_counts()

In [ ]:
min_needed_margin = tdf.margin_cost.max()
print('min needed margin', min_needed_margin)

In [ ]:
(tdf.realized_pnl - tdf.fee).cumsum().plot()

In [ ]:
tdf.amount.plot()

In [ ]:
gain = (realized_pnl_sum - fee_sum + min_needed_margin) / min_needed_margin
n_days = (tdf.timestamp.max() - tdf.timestamp.min()) / 1000 / 60 / 60 / 24
adg = gain ** (1 / n_days)
average_daily_abs_profit = realized_pnl_sum / n_days
print('min_needed_margin', min_needed_margin)
print('realized_pnl_sum', realized_pnl_sum)
print('gain', gain)
print('n_days', n_days)
print('average daily gain', adg)
print('average_daily_abs_profit', average_daily_abs_profit)

In [ ]:
# for testing multiple settings

In [ ]:
rs = {}

In [ ]:
v0s = np.linspace(900, 1100, 3).astype(int)   # ema_spans
v1s = np.linspace(0.0009, 0.0011, 3).round(6) # flashcrash_factors
v2s = np.linspace(0.00145, 0.00155, 1).round(5) # markups
v3s = np.linspace(99, 101, 3).astype(int)     # leverages
v4s = [2]                               # long, shrt or both
print(v0s)
print(v1s)
print(v2s)
print(v3s)
print(v4s)

In [ ]:

for v0 in v0s:
    settings['ema_span'] = v0
    for v1 in v1s:
        settings['flashcrash_factor'] = v1
        for v2 in v2s:
            settings['markup'] = v2
            for v3 in v3s:
                settings['leverage'] = v3
                for v4 in v4s:
                    if v4 == 0:
                        settings['enter_long'] = True
                        settings['enter_shrt'] = False
                    elif v4 == 1:
                        settings['enter_long'] = False
                        settings['enter_shrt'] = True
                    else:
                        settings['enter_long'] = True
                        settings['enter_shrt'] = True
                    key = (v0, v1, v2, v3, v4)
                    if key in rs:
                        print('skipping', key)
                        continue
                    trades, adf = backtest(adf, settings)
                    tdf = pd.DataFrame(trades).set_index('agg_id')
                    result = {'net_pnl': tdf.realized_pnl.sum() - tdf.fee.sum()}
                    result['amount_max'] = tdf.amount.max()
                    result['amount_min'] = tdf.amount.min()
                    result['amount_abs_max'] = tdf.amount.abs().max()
                    result['amount_abs_sum'] = tdf.amount.abs().sum()
                    result['n_trades'] = len(trades)
                    result['margin_cost_max'] = tdf.margin_cost.max()
                    print('\n', key, list(map(lambda x: round(x, 6), result.values())))
                    
                    rs[key] = result                    
                

In [ ]:
rdf = pd.DataFrame(rs).T
rdf = rdf.join(pd.Series((rdf.net_pnl + rdf.margin_cost_max) / rdf.margin_cost_max, name='gain'))
rdfs = rdf.sort_values(['amount_abs_sum', 'gain'], ascending=[True, False])
rdfs.head(30)